In [ ]:
from os import path
import pandas as pd

from src import cumplo_core as cumplo_core
from src import some_utils as utls

In [ ]:
# Jupyter notebooks will cache the scripts,
# but this allows for automatic reloading of updated scripts,
# eliminating the need to manually reload each time.
%load_ext autoreload
%autoreload 2

<div class="alert">
<h5>Import data:</h5>

We will read the flow and movements files to work with them

</div>


In [ ]:
# Data in/out path...
data_in_folder = "./data_in/"
data_out_folder = "./data_out/"

In [ ]:
movs_file_path = utls.get_most_recent_filename(data_in_folder, "Resumen de movimientos - ", "xls")
flows_file_path = utls.get_most_recent_filename(data_in_folder, "Resumen de flujos - ", "xlsx")

In [ ]:
flows_df = pd.read_excel(flows_file_path)
movs_df = pd.read_excel(movs_file_path)

<div class="alert">
<h5>Clean and transform data:</h5>

We will read the flow and movements files to work with them

</div>


In [ ]:
# Flows:

# Remove last 5 rows, that are Label cells
flows_df = flows_df[:-5]

# Convert Ids to strings w/o decimals
flows_df["ID"] = flows_df["ID"].apply(int).apply(str)

In [ ]:
# Movements:

# Fill NAs
movs_df = movs_df.fillna(0)

# Remove rows that Descripción is 'Abono a Saldo Cumplo' or 'Retiro de Saldo Cumplo'
movs_df = movs_df.query(
    'Descripción != "Abono a Saldo Cumplo" & Descripción != "Retiro de saldo Cumplo"'
)

# keep only meaningful movements, Cargo or Abono > 0...
movs_df = movs_df.query("Cargo > 0 | Abono > 0")

<div class="alert">
<h5>Look for RemateID on all movements:</h5>

We are going to create a new column called `Solicitud` , where we are going to extract the 'relevant' info from `Descripción` column.

This is going to be used later to group investments by `RemateID`.

</div>


In [ ]:
# pattern 1: 'solicitud: (\w.+)'
# Liberación de Puntos Cumplo Retenidos, solicitud: Prepago crédito Cumplo contra línea capital de trabajo
# Retención de Puntos Cumplo, solicitud: Prepago crédito Cumplo contra línea capital de trabajo
# Retención de Puntos Cumplo, solicitud: Capital de trabajo Linea Comex
# Devolución de fondos por crédito no concretado, solicitud: Crédito Kio Solutions
# Pago de inversión, solicitud: Crédito Kio Solutions
pattern1 = "solicitud: (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern1, "Descripción", "Solicitud")

# pattern 2: 'solicitud "(\w.+)".'
# Devolución de Puntos por solicitud "Capital de trabajo Linea Comex".
pattern2 = 'solicitud "(\w.+)".'
movs_df = utls.match_group_and_assign(movs_df, pattern2, "Descripción", "Solicitud")

# pattern 3: 'Reajuste puntos Cumplo por solicitud (\w.+)'
# Reajuste puntos Cumplo por solicitud 73278
pattern3 = "Reajuste puntos Cumplo por solicitud (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern3, "Descripción", "Solicitud")

# pattern 4: 'regularizacion saldo cumplo operacion (\w.+)'
# regularizacion saldo cumplo operacion 70500
pattern4 = "regularizacion saldo cumplo operacion (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern4, "Descripción", "Solicitud")

# pattern 5: 'reembolso puntos cumplo operación (\w.+)'
# reembolso puntos cumplo operación 73014
pattern5 = "reembolso puntos cumplo operación (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern5, "Descripción", "Solicitud")

# pattern 6: 'Devolución de fondos por crédito no concretado, solicitud: (\w.+)'
# Devolución de fondos por crédito no concretado, solicitud: Capital de trabajo Linea Comex
pattern6 = "Devolución de fondos por crédito no concretado, solicitud: (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern6, "Descripción", "Solicitud")

# pattern 7: 'regularizacion saldo cumplo (3cuotas) operación (\w.+)'
# regularizacion saldo cumplo (3cuotas) operación 71701
pattern7 = "regularizacion saldo cumplo \(3cuotas\) operación (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern7, "Descripción", "Solicitud")

# pattern 8: 'regularizacion saldo cumplo, capital faltante operación (\w.+)'
# regularizacion saldo cumplo, capital faltante operación 71701
pattern8 = "regularizacion saldo cumplo, capital faltante operación (\w.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern8, "Descripción", "Solicitud")

In [ ]:
# Movements, get the ID of the investment on a new column
# First a 'quick and dirty' approach that works for all 'modern' nomeclature
movs_df["RemateID"] = movs_df["Solicitud"].str.split().str[-1]

In [ ]:
# Set non-numeric RemateID as NA
mask = ~pd.to_numeric(movs_df["RemateID"], errors="coerce").notnull()
movs_df.loc[mask, "RemateID"] = pd.NA

In [ ]:
# Now for the rest of the investments where we weren't able to find its ID,
# we will use the info from flows...

# First we need to know which ids we already have;
known_ids = movs_df.query("RemateID.notna()")["RemateID"].unique()

# And we use that to only go through the unknown ids...
unknown_id_mask = ~flows_df["ID"].isin(known_ids)


for index, row in flows_df[unknown_id_mask].iterrows():
    flow_id = row["ID"]
    flow_solicitud = row["Solicitud"]
    mask = movs_df["Solicitud"].str.startswith(flow_solicitud, na=False)

    if not mask.any():
        # We can't find any !!
        desc = movs_df["Solicitud"]

        print(f"We can't find any match for: Index: [{index}] id:[{flow_id}]- [{flow_solicitud}]")
        continue

    # display(f'we found;  {current_id}  [{index}] - [{current_solicitud}]')
    movs_df.loc[mask, "RemateID"] = str(int(flow_id))

In [ ]:
# Opposite approach;
# Since there are weird edge cases;
# Like, description in flows that has more info than in movements
# our previous search and assignment would fail in some cases;
# For example we have:
# in flows_df Solicitud: "Crédito Más Ingenieria 23028"
# and in movs_df Solicitud:"Crédito Más Ingenieria "
# But for this we will only use the ids that are are still not assigned
known_ids = set(movs_df.query("RemateID.notna()")["RemateID"])
flow_ids = set(flows_df["ID"])
unassigned_flow_ids = flow_ids - known_ids

unnasigned_ids_mask = flows_df["ID"].isin(unassigned_flow_ids)
unassigned_df = flows_df[unnasigned_ids_mask]

# Skip all those movs where we already know the id
na_ids_mask = movs_df["RemateID"].isna()
# Also,s kip all those movs where 'Solicitud' is na
na_solicitus_mask = movs_df["Solicitud"].notna()

# Get ID using data on movs_df
for index, row in movs_df[na_ids_mask & na_solicitus_mask].iterrows():
    current_solicitud = row["Solicitud"]

    mask = unassigned_df["Solicitud"].str.startswith(current_solicitud, na=False)
    if not mask.any():
        # We can't find any !!
        print(f"We can't find any match for: Index: [{index}] - [{current_solicitud}]")
        continue

    current_id = unassigned_df[mask]["ID"].values[0]
    # print(f'we found; for [{index}] - [{current_solicitud}]')
    movs_df.loc[index, "RemateID"] = str(int(current_id))

<div class="alert">
<h5>Extract Actors:</h5>

On `Solicitud` column most of the time we have enough info to extract the `Actor` behind the operation.

This would be useful later to get the RemateID.

</div>


In [ ]:
# Extract Actor names from Solicitud

# pattern1: 'Credito (.+) (\d+)' # No accent mark!
# Credito COMERCIAL 2050 SPA 24494 >> 'COMERCIAL 2050 SPA'
# Credito INMOBILIARIA JUAN CARLOS VALDEBENITO ORTIZ E.I.R.L 23356, parte II >> 'INMOBILIARIA JUAN CARLOS VALDEBENITO ORTIZ E.I.R.L'
pattern1 = "Credito (.+) (\d+)"
movs_df = utls.match_group_and_assign(movs_df, pattern1, "Solicitud", "Actor")

# pattern2: '^Crédito (.+)' # Accent mark!
# Crédito eHS SpA. >> 'eHS SpA.' # tilde
# Crédito Notebookcenter >> 'Notebookcenter'
# Crédito Buscalibre.com >> 'Buscalibre.com'
# Crédito Corto Plazo II: Renovación de línea de construcción >> 'Corto Plazo II: Renovación de línea de construcción'
pattern2 = "^Crédito (.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern2, "Solicitud", "Actor")

# pattern3: '^Credito (.+)' # No accent mark (and no investment ID at the end...)
# Credito Green Logistic
# Credito Ambrosio Torresilla
pattern3 = "^Credito (.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern3, "Solicitud", "Actor")

# pattern4: '(.+): .*'
# BAXIS EIRL: Crédito empresa 80% garantizado >> 'BAXIS EIRL'
# INMOBILIARIA JUAN CARLOS VALDEBENITO ORTIZ E.I.R.L: Crédito Nº2 empresa Cero Cupón 100% garantizado SuAval >> 'INMOBILIARIA JUAN CARLOS VALDEBENITO ORTIZ E.I.R.L'
pattern4 = "(.+): .*"
movs_df = utls.match_group_and_assign(movs_df, pattern4, "Solicitud", "Actor")

# Now we have just all the rest, but avoid the 'only numbers'
# pattern5: '^(?!\d+$)(.+)'
#   ^(?!\d+$) => Do not match numbers until the end...
#   (.+) => but we want all the other options...
# 22474 - Solicitud Empresa >> '22474 - Solicitud Empresa'
# Financiamento Febrero >> 'Financiamento Febrero'
# 73278 >> na
pattern5 = "^(?!\d+$)(.+)"
movs_df = utls.match_group_and_assign(movs_df, pattern5, "Solicitud", "Actor")

In [ ]:
# Clean and replace spanish characters...
movs_df["Actor"] = movs_df["Actor"].apply(utls.clean_spanish_characters)

In [ ]:
# We fill the pending NA Actors
# We have a 'complete' dframe of non-na Actors
complete_df = movs_df.query("Actor.notna() & RemateID.notna()")
# We create a dictionary RemateID -> Actor
dict_id_acts = dict(zip(complete_df["RemateID"], complete_df["Actor"]))
# And we fill that using map
movs_df["Actor"] = movs_df["Actor"].fillna(
    movs_df["RemateID"].map(dict_id_acts, na_action="ignore")
)

In [ ]:
# And the last RemateID filling!
# When RemateID is NA and Actor is not na, fill with Actor!

# This works for the old-old investments
movs_df["RemateID"] = movs_df["RemateID"].fillna(movs_df["Actor"])

<div class="alert">
<h5>Explore data:</h5>

Explore all the Investments, grouped by RemateID

</div>


In [ ]:
cumplo_core.explore_by_id(movs_df)

<div class="alert">
<h5>Fix data:</h5>

For some (another) weird reason, there are some investments that aren't completed on the movements flow.
One way to spot them is to explore and check all those that have negative earnings.

Then we will find:</br>

- `a`. late, uncollectible, or active investments</br>
- `b`. weird investment that must be fixed!</br>

For `a` there is nothing that we need to do, but for `b` we have to apply a fix.

It seems like the cause behind this is, cumplo has some issues exporting movements when there is more than one payment on the same date, or if the investment was cancelled, or others.

I wasn't able to figure out a set of rules, so I had to inspect on the website (and sometimes check agains my bank account) to see what was going on.

</div>


In [ ]:
negative_earning_ids = cumplo_core.find_negative_earning_ids(movs_df)
print(f"We found [{len(negative_earning_ids)}] investments with a negative balance")

In [ ]:
# Explore negative investments; a.-late, uncollectible, active, or b.- incorrectly-recorded!
# We are looking for the b.- incorrectly-recorded ids...
cumplo_core.explore_by_id(movs_df, negative_earning_ids)

<div class="alert">
<h5>Fix data:</h5>

After looking for those special cases we come up with the following fix:

For each one of these investments, we are going to add a new row with a `Abono` that fixes the total earnings.
And we will store that data in a csv file called `fix_data.csv` located on `./data_in/`.

---

In my case, I had to check on cumplo.cl for each `weird` investment Id and see if the total earnings were equal to the actual earnings (or `Monto Recibido` ). When not, I calculated the difference and used it as an extra `Abono` to match the actual amount.

For the date of the fix, I used the deposit date that you can find going to the investment's details (`Resumen de Pagos`).

So at the end, I came up with a `fix_data.csv` similar to this:

```csv
    RemateID,Actor,Date_YYYY-MM-DD,Abono,Cargo
    27555,avanza construccion ltda,2016-Sep-05,35409,0
    27761,Comercial 2050,2016-Nov-11,9869,0
    28106,avendaño y aranda,2016-May-05,52636,0
    ...
    33728,dolphins,2017-11-14,7700,0
    ...
    66019,Comercial 2050,2023-01-19,324095,0
```

</div>


In [ ]:
# Apply fix!
fixdata_csv_path = path.join(data_in_folder, "fix_data.csv")
movs_df = cumplo_core.insert_fix(movs_df, fixdata_csv_path)

In [ ]:
# Clean and replace spanish characters...
movs_df["Actor"] = movs_df["Actor"].apply(utls.clean_spanish_characters)

<div class="alert">
<h5>Classify data:</h5>

Now we are going to clasiffy the investments in 4 categories;

- `a`. Unexecuted (Investments that never were active. Funds were returned.)
- `b`. Completed (Investments that are finish)
- `c`. Active (Investments on track :), also investment with late payments but that are not older that some grace period...)
- `d`. Uncollectible (Investments with pending payments that are older than some grace period...)

We will get the ids of investments that are active, late, and uncollectible, directly from the flows excel file.
Using the text color on the flows we can determine when a flow is late, expected, on-time, late-but-payed, etc...

</div>


<div class="alert">
<h5>Classify active, late, and uncollectible:</h5>

From the flows file we are going to extract the investments that are active, late, or uncollectible.

Uncollectible will be all investments that are late more than `grace_period_days`.

Also we will retrieve all the IDs that are not present in the flow file but exists in Movements.

(If a investment was cancelled, or if a investment is too recent, the confirmation is pending this could be the case!)

</div>


In [ ]:
# We'll use grace_period_days of 60 days (2 months)
grace_period_days = 60
flow_ids, active_ids, late_ids, uncollectible_ids = cumplo_core.extract_active_and_late_ids(
    flows_file_path, grace_period_days
)

# Obtain all the ids that are not present in the flow file
all_ids = movs_df["RemateID"].unique()
not_present_in_flows_ids = set(all_ids) - set(flow_ids)

# Uncomment the next line; if you want to explore the ids that are not present in flows but in movements
# We expect a lot of old and cancelled, and a few super recent investments.
# The if Cargos-Abonos is close to zero, then we think that is a cancelled investment
cumplo_core.explore_by_id(movs_df, not_present_in_flows_ids)

<div class="alert">
<h5>Classify Just Payed:</h5>

From the flows file we are going to extract the investments that are just payed (but not confirmed).

For that we consider all of them that a.- Are not present in flow files, and b.- the difference between cargos and abonos is a considerable negative amount.

</div>


In [ ]:
# At this point, investments that are 'Abonos' - 'Cargos' are a considerable
# negative number and is not present in flows file can be considered as 'just payed'.

# Instead of zero, we sill set a considerable_amount = 100000, which seems to be a good number...
considerable_amount = 100000

just_payed_ids = cumplo_core.extract_just_payed(
    movs_df, not_present_in_flows_ids, considerable_amount
)

# Uncomment the next line; if you want to explore the ids classified as unexecuted
cumplo_core.explore_by_id(movs_df, just_payed_ids)

<div class="alert">
<h5>Classify Unexecuted:</h5>

From the flows file we are going to extract the investments that weren't executed.

For that we consider all of them where the difference between cargos and abonos is a despreciable amount.
Taking into consideration the ids that are not registered on flows.

For the just paye

</div>


In [ ]:
# At this point, investments that are 'Abonos' - 'Cargos' are zero (or close to zero)
# can be considered as unexecuted.

# Instead of zero, we sill set a despreciable_amount of 200, which seems to be a good number...
despreciable_amount = 200
unexecuted_ids = cumplo_core.extract_unexecuted(movs_df, despreciable_amount)

# Uncomment the next line; if you want to explore the ids classified as unexecuted
cumplo_core.explore_by_id(movs_df, unexecuted_ids)

<div class="alert">
<h5>Remove unexecuted from extracted lists:</h5>

We update our previous findings removing all the unexecuted ids.

</div>


In [ ]:
# Filter ids removing unexecuted ids
active_ids = list(set(active_ids) - set(unexecuted_ids))
late_ids = list(set(late_ids) - set(unexecuted_ids))
uncollectible_ids = list(set(uncollectible_ids) - set(unexecuted_ids))

# Uncomment the next lines; if you want to explore the ids classified as active_ids, late_ids, and uncollectible_ids
# cumplo_core.explore_by_id(movs_df, active_ids)
# cumplo_core.explore_by_id(movs_df,late_ids)
# cumplo_core.explore_by_id(movs_df, uncollectible_ids)

<div class="alert">
<h5>Completed investments:</h5>

All the ids that aren't `active`, `unexecuted`, `just_payed` or `late_but_collectibles` will be considered as `completed`

</div>


In [ ]:
# Now we will state that all ids not active, unexecuted or late_but_collectibles are completed ids:
late_but_collectibles = set(late_ids) - set(uncollectible_ids)

# beware, we consider the uncollectibles as 'completed'. Cumplo.cl doesn't do this.
completed_ids = (
    set(all_ids)
    - set(active_ids)
    - set(unexecuted_ids)
    - set(just_payed_ids)
    - set(late_but_collectibles)
)

# Uncomment the next line; if you want to explore the ids classified as completed_ids
# cumplo_core.explore_by_id(movs_df, completed_ids)

<div class="alert">
<h5>Adjustments; Set Completed but not completely payed as Uncollectibles:</h5>

</div>


In [ ]:
completed_mask = movs_df["RemateID"].isin(completed_ids)
completed_df = movs_df[completed_mask]

grace_period_days_since_last_payment = 60
completed_but_uncollectible_ids = cumplo_core.extract_uncollectibles(
    completed_df, grace_period_days_since_last_payment
)

# Remove from completed
completed_ids = list(set(completed_ids) - set(completed_but_uncollectible_ids))
# Add to uncollectible_ids
uncollectible_ids = list(set(uncollectible_ids) | set(completed_but_uncollectible_ids))

<div class="alert">
<h5>Some stats:</h5>

</div>


In [ ]:
# Stats;
print(
    f"Total of: [{len(all_ids):,}], Unexecuted: [{len(unexecuted_ids)}], Completed: [{len(completed_ids):,}]"
)
print(
    f"Active: [{len(active_ids)}], Just Payed: [{len(just_payed_ids)}], Late: [{len(late_ids)}], Uncollectibles: [{len(uncollectible_ids)}]"
)
print(f"Late in the grace_period: [{(len(late_ids) - len(uncollectible_ids))}]")

<div class="alert">
<h5>Assign classification:</h5>

We will create a new column called `Estado` to assign the classification of each investment.

</div>


In [ ]:
movs_df["Estado"] = "NotAssigned"

# Unexecuted
unexecuted_mask = movs_df["RemateID"].isin(unexecuted_ids)
movs_df.loc[unexecuted_mask, "Estado"] = "Unexecuted"

# Completed
completed_mask = movs_df["RemateID"].isin(completed_ids)
movs_df.loc[completed_mask, "Estado"] = "Completed"

# Active: Active, Just Payed, or Late => Late are late, but Active!
active_mask = movs_df["RemateID"].isin(active_ids)
late_mask = movs_df["RemateID"].isin(late_ids)
just_payed_mask = movs_df["RemateID"].isin(just_payed_ids)
movs_df.loc[late_mask | active_mask | just_payed_mask, "Estado"] = "Active"

# Uncollectible
uncollectible_mask = movs_df["RemateID"].isin(uncollectible_ids)
movs_df.loc[uncollectible_mask, "Estado"] = "Uncollectible"

<div class="alert">
<h5>Save data:</h5>

Export dataframe to `data_out_folder`. <br>
In our case: `./data_out/sanitized_and_classified.feather`

</div>


In [ ]:
output_file_path = path.join(data_out_folder, "sanitized_and_classified.feather")
movs_df.to_feather(output_file_path)